# Task definition
Implement LSTM Sentiment Tagger for imdb reviews dataset.

1. (5pt) Fill missing code below
    * 1pt implement vectorization
    * 2pt implement \_\_init\_\_ and forward methods of models
    * 2pt implement collate function
2. (4pt) Implement training loop, choose proper loss function, use clear ml for max points.
    * 2pts is a baseline for well written, working code
    * 2pts if clear ml used properly
3. (3pt) Train the models (find proper hyperparams). Make sure you are not overfitting or underfitting. Visualize training of your best model (plot training, and test loss/accuracy in time). Your model should reach at least 87% accuracy. For max points it should exceed 89%.
    * 1pt for accuracy above 89%
    * 1pt for accuracy above 87%
    * 1pt for visualizations

Remarks:
* Use embeddings of size 50
* Use 0.5 threshold when computing accuracy.
* Use supplied dataset for training and evaluation.
* You do not have to use validation set.
* You should monitor overfitting during training.
* For max points use clear ml to store and manage logs from your experiments.
* We encourage to use pytorch lightning library (Addtional point for using it - however the sum must not exceed 12)

[Clear ML documentation](https://clear.ml/docs/latest/docs/)

[Clear ML notebook exercise from bootcamp](https://colab.research.google.com/drive/1wtLb4gg8beLS7smcyJlOZppn6_rQvSxL?usp=sharing)

In [ ]:
!pip install clearml

import os
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torchtext
from clearml import Task

import torch
from torch import nn
from torch import optim

from torch.utils.data import Dataset, DataLoader

from collections import Counter

In [ ]:

web_server = 'https://app.community.clear.ml'
api_server = 'https://api.community.clear.ml'
files_server = 'https://files.community.clear.ml'
access_key = '2M0M97GRKONAW4PLA9VY'#@param {type:"string"}
secret_key = 'EbPYvpD36l8NhA4R5F9MRL8HP9n9ae8SuVZ0IMfjGhgbeg3Ccm'#@param {type:"string"}

Task.set_credentials(web_host=web_server,
                     api_host=api_server,
                     files_host=files_server,
                     key=access_key,
                     secret=secret_key)

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1hK-3iiRPlbePb99Fe-34LJNZ5yB-nduq
!tar -xvzf imdb_dataset.gz
data = pd.read_csv("imdb_dataset.csv")

Downloading...
From: https://drive.google.com/uc?id=1hK-3iiRPlbePb99Fe-34LJNZ5yB-nduq
To: /content/imdb_dataset.gz
100% 77.0M/77.0M [00:00<00:00, 242MB/s]
imdb_dataset.csv


In [ ]:
PADDING_VALUE = 0
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class NaiveVectorizer:
    def __init__(self, tokenized_data, **kwargs):
        """Converts data from string to vector of ints that represent words.
        Prepare lookup dict (self.wv) that maps token to int. Reserve index 0 for padding.
        """
        tokenized_data = [seq.split() for seq in tokenized_data]
        ### Your code goes here ###
        flat_list = [item for sublist in tokenized_data for item in sublist]
        counter = Counter(flat_list)
        the_best = counter.most_common(3000)

        d = {'padding': 0}
        for i, (tok,_) in enumerate(the_best):
            d[tok] = i + 1

        self.wv = d

        ##################################

    def vectorize(self, tokenized_seq):
        """Converts sequence of tokens into sequence of indices.
        If the token does not appear in the vocabulary(self.wv) it is ommited
        Returns torch tensor of shape (seq_len,) and type long."""
        ### Your code goes here ###
        seqli = []
        for tok in tokenized_seq:
          if tok in self.wv:
            seqli.append(self.wv[tok])
          else:
            continue
          seq = torch.LongTensor(seqli)

        return seq

        ##################################

class ImdbDataset(Dataset):
    SPLIT_TYPES = ["train", "test", "unsup"]

    def __init__(self, data, preprocess_fn, split="train"):
        super(ImdbDataset, self).__init__()
        if split not in self.SPLIT_TYPES:
            raise AttributeError(f"No such split type: {split}")

        self.split = split
        self.label = [i for i, c in enumerate(data.columns) if c == "sentiment"][0]
        self.data_col = [i for i, c in enumerate(data.columns) if c == "tokenized"][0]
        self.data = data[data["split"] == self.split]
        self.preprocess_fn = preprocess_fn

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.preprocess_fn(self.data.iloc[idx, self.data_col].split())
        label = self.data.iloc[idx, self.label]
        return (seq, label)

naive_vectorizer = NaiveVectorizer(data.loc[data["split"] == "train", "tokenized"])

def get_datasets():
    train_dataset = ImdbDataset(data, naive_vectorizer.vectorize)
    test_dataset = ImdbDataset(data, naive_vectorizer.vectorize, split="test")

    return train_dataset, test_dataset

def custom_collate_fn(pairs):
    """This function is supposed to be used by dataloader to prepare batches
    Input: list of tuples (sequence, label)
    Output: sequences_padded_to_the_same_lenths, original_lenghts_of_sequences, lables.
    torch.nn.utils.rnn.pad_sequence might be usefull here
    """
    ### Your code goes here ###
    lengths, labels, seq = [], [], []
    for s, l in pairs:
      seq.append(s)
      labels.append(l)
      lengths.append(len(s))
    seqcs = torch.nn.utils.rnn.pad_sequence(seq, batch_first=True)
    seqcs = torch.LongTensor(seqcs).to(DEVICE)
    labels = torch.FloatTensor(labels).to(DEVICE)
    lengths = torch.LongTensor(lengths)
    #################################
    return seqcs, lengths, labels

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

cuda


In [ ]:
"""Implement LSTMSentimentTagger.
The model should use a LSTM module.
Use torch.nn.utils.rnn.pack_padded_sequence to optimize processing of sequences.
When computing vocab_size of embedding layer remeber that padding_symbol counts to the vocab.
Use sigmoid activation function.
"""
class LSTMSentimentTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, classes):
        super(LSTMSentimentTagger, self).__init__()
        self.hidden_dim = hidden_dim

        ### Your code goes here ###
        self.embedding_dim = nn.Embedding(vocab_size, embedding_dim).to(DEVICE)
        self.vocab_size = vocab_size
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, proj_size = int(embedding_dim / 2)).to(DEVICE)
        self.fc = nn.Linear(int(embedding_dim / 2),1).to(DEVICE)
        self.dropout = nn.Dropout(0.5)
        #################################

    def forward(self, sentence, lengths):
        ### Your code goes here ###
        x = sentence
        #x[x > self.vocab_size] = 0
        x = self.embedding_dim(x)
        x = self.dropout(x)
        x = torch.nn.utils.rnn.pack_padded_sequence(x, lengths.cpu().numpy(), batch_first = True, enforce_sorted= False).to(DEVICE)
        x,_ = self.lstm(x)
        x,_ = torch.nn.utils.rnn.pad_packed_sequence(x)
        x = torch.stack([x[lengths[i]-1, i, :] for i in range(len(sentence))])
        #x = self.dropout(x)
        x = self.fc(x)
        x = torch.sigmoid(x)
        x = x.squeeze()
        scores = x
        #################################
        return scores

# Trainig loop and visualizations


In [ ]:
config = {
  'lr': 0.005,
  'hidden_dim': 500,
  'len_dic' : 3000,
  'dropout' : 0.5
}
task = Task.create(project_name='Homework3', task_name='auto_log_experiment')

task.mark_started()
logger = task.get_logger()
task.connect(config)



def train_loop(data, model, loss_fn, optimizer, batch_size):

    model.train()
    train_loss, correct = 0, 0
    for i in range(0, len(data) - batch_size, batch_size):
        data.data = data.data.iloc[torch.randperm(data.data.shape[0])]
        # Compute prediction and loss
        seqcs, lengths, labels = custom_collate_fn([data[i+j] for j in range(batch_size)])

        pred = model(seqcs, lengths)
        loss = loss_fn(pred, labels)
        pred = (pred > 0.5)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred == labels).type(torch.float).sum().item()
    correct /= len(data)
    train_loss /= len(data)

    logger.report_scalar(title='Train Loss', series='Train', iteration=t, value=loss)
    logger.report_scalar(title='Train Accuracy', series='Train', iteration=t, value=correct*100)

    print(f"train_loss: {train_loss:>7f}, train_acc: {correct}")
    return correct, train_loss

def test_loop(data, model, loss_fn, batch_size):
    model.eval()
    num_batches = len(data)
    test_loss, correct = 0, 0
    for i in range(0, len(data) - batch_size, batch_size):
       seqcs, lengths, labels = custom_collate_fn([data[i+j] for j in range(batch_size)])
       with torch.no_grad():
          pred = model(seqcs, lengths)
          loss = loss_fn(pred, labels)
          pred = (pred > 0.5)
          correct += (pred == labels).type(torch.float).sum().item()
       test_loss += loss.item()

    test_loss /= len(data)
    correct /= len(data)

    logger.report_scalar(title='test Loss', series='Train', iteration=t, value=test_loss)
    logger.report_scalar(title='Test Accuracy', series='Test', iteration=t, value=correct*100)
    print(f"test_loss: {test_loss:>7f}, test_acc: {correct}")
    return correct, test_loss

In [ ]:
train_data, test_data = get_datasets()

In [ ]:
TRAIN_ACC = []
TRAIN_LOSS = []
TEST_ACC = []
TEST_LOSS = []


model = LSTMSentimentTagger(embedding_dim = 50, hidden_dim=500, vocab_size = len(naive_vectorizer.wv), classes=2)
loss_fn = nn.MSELoss()
#loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

epochs = 8
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    tr_a, tr_l = train_loop(train_data, model, loss_fn, optimizer, batch_size = 50)
    te_a, te_l = test_loop(test_data, model, loss_fn, batch_size = 50)
    TRAIN_ACC.append(tr_a)
    TRAIN_LOSS.append(tr_l)
    TEST_ACC.append(te_a)
    TEST_LOSS.append(te_l)
print("Done!")

Epoch 1
-------------------------------
train_loss: 0.004641, train_acc: 0.59452
test_loss: 0.003255, test_acc: 0.76708
Epoch 2
-------------------------------
train_loss: 0.002760, train_acc: 0.80524
test_loss: 0.002125, test_acc: 0.85108
Epoch 3
-------------------------------
train_loss: 0.002146, train_acc: 0.85124
test_loss: 0.001884, test_acc: 0.86824
Epoch 4
-------------------------------
train_loss: 0.001871, train_acc: 0.872
test_loss: 0.001809, test_acc: 0.87704
Epoch 5
-------------------------------
train_loss: 0.001793, train_acc: 0.87696
test_loss: 0.001745, test_acc: 0.87784
Epoch 6
-------------------------------
train_loss: 0.001664, train_acc: 0.88676


In [ ]:
import plotly.express as px

fig = px.line(x = range(len(TRAIN_ACC)), y = TRAIN_ACC, title = 'TRAIN_ACC')
fig.show()

In [ ]:
fig = px.line(x = range(len(TRAIN_LOSS)), y = TRAIN_LOSS, title = 'TRAIN_LOSS')
fig.show()

In [ ]:
fig = px.line(x = range(len(TEST_ACC)), y = TEST_ACC, title = 'TEST_ACC')
fig.show()

In [ ]:
fig = px.line(x = range(len(TEST_LOSS)), y = TEST_LOSS, title = 'TEST_LOSS')
fig.show()